In [1]:
import os

import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq

dataDir = "Data"
genomeTreeDir = "Genome_tree"

if not os.path.exists(genomeTreeDir):
    os.mkdir(genomeTreeDir)

In [16]:
df = pd.read_csv(os.path.join(dataDir, "meta_data.csv"))
df["Collection date"] = pd.to_datetime(df["Collection date"])

In [33]:
df["Area"] = pd.Series()
df["Location"] = pd.Series()
for index, row in df.iterrows():
    if row["Country"] == "China":
        area = row["Province"]
        if area == "Guandong":
            area = "Guangdong"
        elif area == "Chongqinq":
            area = "Chongqing"
        location = row["City"]
        if pd.isna(location):
            print("location")
    else:
        area = row["Country"]
        location = row["Country"]
    df.loc[index, "Area"] = area
    df.loc[index, "Location"] = location

df[["Accession ID", "Virus name", "Location", "Area", "Country", "Collection date"]].to_csv(
    os.path.join(dataDir, "info.csv"),
    sep=",",
    index=False
)

location
location


In [34]:
metaDir = os.path.join(dataDir, "ByArea")

if not os.path.exists(metaDir):
    os.mkdir(metaDir)

for area in df["Area"].drop_duplicates():
    df[df["Area"] == area].to_csv(os.path.join(metaDir, area + ".csv"))

In [44]:
outgroup = None
earlies = None
outputSeqs = []

# excluded as agreed http://virological.org/t/phylodynamic-analysis-90-genomes-12-feb-2020/356/4
excluded = ["EPI_ISL_406592", "EPI_ISL_406595"]

outgroup = "EPI_ISL_402131"

for record in SeqIO.parse(os.path.join(dataDir, "genomes.fasta"), "fasta"):
    ac = record.id
    if ac not in excluded:
        outputSeqs.append(record)

with open(os.path.join(genomeTreeDir, "outgroup.txt"), 'w') as f:
    f.write(outgroup + "\n")
SeqIO.write(outputSeqs, os.path.join(genomeTreeDir, "raw.fasta"), "fasta")

166